<a href="https://colab.research.google.com/github/joaovitormelo/ProjetosML/blob/main/WinesClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [1]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import SVC

#Carregando os dados

In [2]:
pathData = 'drive/MyDrive/Colab Notebooks/SupervisedLearning'

In [3]:
#Carrega o dataset
df = pd.read_csv(pathData + '/wines.csv')

#Pré-processando o Dataset

In [4]:
#Filtra apenas colunas relevantes
df = df[['description', 'variety']]

In [5]:
#Cria objeto Counter que recebe uma lista dos valores presentes na coluna "variety" (mantendo as repetições/ordem)
#São meus labels
counter = Counter(df['variety'].tolist())

In [6]:
#Obtém os mais comuns (list of tuples {name, index in list}) e os enumera
enumeration = enumerate(counter.most_common(10))
#Obtém um dict {name: number[0-9] da enumeração}
#Esse número será como eu irei representar cada label, num formato numérico
top_10_varieties = {wine[0]: idx for idx, wine in enumeration}
print(top_10_varieties)

{'Pinot Noir': 0, 'Chardonnay': 1, 'Cabernet Sauvignon': 2, 'Red Blend': 3, 'Bordeaux-style Red Blend': 4, 'Riesling': 5, 'Sauvignon Blanc': 6, 'Syrah': 7, 'Rosé': 8, 'Merlot': 9}


In [7]:
'''
Obtém uma série de booleans correspondentes às rows que serão ou não incluídas
dado o valor de "variety": deixa só as mais comuns
'''
filterVariety = df['variety'].map(lambda x: x in top_10_varieties)
#Aplica o filtro de rows no df
df = df[filterVariety]

In [8]:
#obtém a lista dos inputs (nesse caso, as descriptions)
description_list = df['description'].tolist()
#obtém a lista dos labels: as variedades à qual os "varietal" (
#vinhos pertencentes a uma certa variedade) pertencem, na forma
#numérica
varietal_list = [top_10_varieties[var] for var in df['variety'].tolist()]
#Converte essa lista para um array do numpy
varietal_list = np.array(varietal_list)

In [9]:
#Aplica a técnica Bag of Words sobre a lista de descrições (nossos inputs)
#Converte uma coleção de documentos de texto em uma matriz de contagem de tokens
#(thus counts)
#token = instance of sequence of characters grouped together
count_vect = CountVectorizer()
x_train_counts = count_vect.fit_transform(description_list)

In [10]:
#Calcula os TF-IDF dos documentos (relevância de cada token)
#Recebe como entrada os counts (bags of words) e retorna os valores processados
#com os TF-IDF
tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)

#Utilidades para os Models

In [11]:
#Realiza cross-validation no meu conjunto de inputs (processed in tf-idf format)
#e labels (varietal list)
train_x, test_x, train_y, test_y = train_test_split(x_train_tfidf, varietal_list, test_size=0.3)

In [12]:
#Função para calcular precisão do modelo: nº de acertos / nº de samples do teste
def getAccuracy():
  n_right = 0
  for i in range(len(y_score)):
    if y_score[i] == test_y[i]:
      n_right+=1
  return (n_right/float(len(test_y))) * 100

#Modelo com Naive Bayes

In [13]:
#Cria o modelo de Naive Bayes multinomial e o treina com o set de treinamento
clf = MultinomialNB().fit(train_x, train_y)
#Obtém as predições do modelo no set de testes
y_score = clf.predict(test_x)

In [14]:
print("Accuracy: %.2f%%" % getAccuracy())

Accuracy: 62.55%


#Modelo com Deep Learning

##Funções de pré-processamento

In [32]:
#Faz os imports para as funções
from nltk import word_tokenize
from collections import defaultdict

In [33]:
#Break each text document of the corpus into tokens, saving the count of that
#token in the dict count. Then, sorts the dict in decrescent order, and return
#a list of the top_x words (skipping the first top_n)
def count_top_x_words(corpus, top_x, skip_top_n):
    count = defaultdict(lambda: 0)
    for c in corpus:
        for w in word_tokenize(c):
            count[w] += 1
    count_tuples = sorted([(w, c) for w, c in count.items()], key=lambda x: x[1], reverse=True)
    return [i[0] for i in count_tuples[skip_top_n: skip_top_n + top_x]]

In [34]:
#Turns the corpus into a matrix, in which each row is a text document, and each
#column is a number representing a token. Also returns the dict that maps
#each token to its number.
def replace_top_x_words_with_vectors(corpus, top_x):
    topx_dict = {top_x[i]: i for i in range(len(top_x))}

    return [
        [topx_dict[w] for w in word_tokenize(s) if w in topx_dict]
        for s in corpus
    ], topx_dict

In [35]:
#Apply both functions. The general idea is to focus on the common words in the corpus,
#and also filter out the most common ones (such as "a", "the", "is", etc)
def filter_to_top_x(corpus, n_top, skip_n_top=0):
    top_x = count_top_x_words(corpus, n_top, skip_n_top)
    return replace_top_x_words_with_vectors(corpus, top_x)

##Imports das features de Deep Learning

In [19]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.utils import to_categorical
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

##Importa e processa dataset novamente

In [20]:
#Carrega o dataset
df = pd.read_csv(pathData + '/wines.csv')

In [21]:
#Obtém novamente as top 10 variedades
counter = Counter(df['variety'].tolist())
top_10_varieties = {i[0]: idx for idx, i in enumerate(counter.most_common(10))}
#Filtra o dataset para ter apenas a rows que contenham as top 10
df = df[df['variety'].map(lambda x: x in top_10_varieties)]

##Obtém Inputs e Labels

In [22]:
#Pega lista das descriptions (inputs)
description_list = df['description'].tolist()
#Converte o corpus da input para uma matrix numérica de tokens
mapped_list, word_list = filter_to_top_x(description_list, 2500, 10)

In [30]:
#Pega lista dos labels (varietals) e converte para a codificação numérica
varietal_list_o = [top_10_varieties[i] for i in df['variety'].tolist()]
#Converte vetor de labels para matriz de binários, em que cada linha é um
#varietal (um label) e cada coluna é uma "classe" (um dos valores possíveis para o label),
#sendo que apenas uma será '1' (a que corresponder à classe do label dessa linha)
varietal_list = to_categorical(varietal_list_o)

In [24]:
#Creio que seja uma constante do dataset, o tamanho máximo que uma review pode ter
max_review_length = 150
#Aplica paddings para preencher com '0's no início de cada linha de mapped_list, a fim de que todas elas tenham o mesmo nº de colunas
mapped_list = sequence.pad_sequences(mapped_list, maxlen=max_review_length)

In [25]:
#Cross Validation
train_x, test_x, train_y, test_y = train_test_split(mapped_list, varietal_list, test_size=0.3)

##Cria e Treina Modelo

In [26]:
#Determina nº de dimensões do vetor de word embedding
embedding_vector_length = 64
#Cria o modelo sequencial
model = Sequential()

In [27]:
#ADIÇÃO DAS CAMADAS
#Então é assim. Com o Keras, você cria models sequenciais e pode adicionar
#layers à vontade.
#Cria layer de word embeddings
model.add(Embedding(2500, embedding_vector_length, input_length=max_review_length))
#Cria uma convolutional layer, com 50 filtros
#e kernel = 5 (tamanho da janela de convolução)
model.add(Conv1D(50, 5))
#Layer que passa o resultado para 1 única dimensão (creio)
model.add(Flatten())
#Layer densa com 100 neurônios
model.add(Dense(100, activation='relu'))
#Camada de output, creio eu, com o número de neurônios igual ao número de classes da saída
#A ativação softmax transforma a saída em um distribuição de probabilidades
model.add(Dense(max(varietal_list_o) + 1, activation='softmax'))

In [28]:
#Configura o treinamento do modelo. Define a loss function, e outros parâmetros
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#Treina o modelo em batchs de tamanho 64 (por atualização de gradiente).
#epochs = iterações em todo o conjunto de input e labels
model.fit(train_x, train_y, epochs=3, batch_size=64)

Epoch 1/3
781/781 [==============================] - 40s 49ms/step - loss: 1.0464 - accuracy: 0.6310
Epoch 2/3
781/781 [==============================] - 35s 45ms/step - loss: 0.6265 - accuracy: 0.7875
Epoch 3/3
781/781 [==============================] - 37s 47ms/step - loss: 0.5249 - accuracy: 0.8201


##Testes

In [29]:
#Realiza as predições no set de teste
y_score = model.predict(test_x)
#Como a última layer tem ativação softmax, de distribuição de probabilidades,
#converte isso para 0 ou 1, atribuindo à máxima probabilidade o 1, e às outras
#o valor 0
y_score = [[1 if i == max(sc) else 0 for i in sc] for sc in y_score]

669/669 [==============================] - 5s 7ms/step


In [31]:
#Contabiliza o nº de acertos para calcular a precisão
n_right = 0
for i in range(len(y_score)):
  if all(y_score[i][j] == test_y[i][j] for j in range(len(y_score[i]))):
    n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(test_y)) * 100)))

Accuracy: 76.45%
